In [1]:
import pandas as pd
import tensorflow.keras as k
from sklearn.preprocessing import StandardScaler

In [3]:
adjective_data = pd.read_csv('./data/ngram_dependencies_adjectives_wide_normalized.csv')
print(adjective_data.columns[:10])

X = adjective_data.iloc[:, 2:]
X = StandardScaler().fit_transform(X)

Index(['dependency', 'calendar_year', '1', '10', '11', '12', '13', '14', '15',
       '16'],
      dtype='object')


In [4]:

def get_dense_relu(x): return k.layers.Dense(x, activation="relu")  #, kernel_regularizer="l2")

In [5]:
n_vars = X.shape[1]

model = k.Sequential([
    k.layers.Dense(500, input_shape=[n_vars], activation="relu"),
    get_dense_relu(200),
    get_dense_relu(50),
    get_dense_relu(10),
    k.layers.Dense(2, name="encoded", activation="sigmoid"),
    get_dense_relu(10),
    get_dense_relu(50),
    get_dense_relu(200),
    k.layers.Dense(n_vars)
])
model.compile(optimizer="adam", loss="mse")
# model.summary()

callback_save_checkpoint = k.callbacks.ModelCheckpoint(
    './data/model_checkpoints/Epoch{epoch:02d}-Loss{loss:.2f}.hdf5',
    monitor='loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

early_stopping = k.callbacks.EarlyStopping(
    "loss",
    min_delta=0.0001,
    patience=3
)

In [6]:
history = model.fit(X, X, batch_size=100, epochs=1000, shuffle=True, callbacks=[early_stopping])  # , callbacks=[callback_save_checkpoint]

Epoch 1/1000
1326/1326 [==============================] - 7s 5ms/step - loss: 0.9426
Epoch 2/1000
1326/1326 [==============================] - 7s 6ms/step - loss: 0.8953
Epoch 3/1000
1326/1326 [==============================] - 7s 5ms/step - loss: 0.8707
Epoch 4/1000
1326/1326 [==============================] - 9s 7ms/step - loss: 0.8532
Epoch 5/1000
1326/1326 [==============================] - 10s 7ms/step - loss: 0.8406
Epoch 6/1000
1326/1326 [==============================] - 13s 10ms/step - loss: 0.8302
Epoch 7/1000
1326/1326 [==============================] - 7s 6ms/step - loss: 0.8223
Epoch 8/1000
1326/1326 [==============================] - 13s 10ms/step - loss: 0.8159
Epoch 9/1000
1326/1326 [==============================] - 21s 16ms/step - loss: 0.8100
Epoch 10/1000
1326/1326 [==============================] - 14s 11ms/step - loss: 0.8052
Epoch 11/1000
1326/1326 [==============================] - 7s 5ms/step - loss: 0.8011
Epoch 12/1000
1326/1326 [=============================

In [7]:
k.models.save_model(model, "./data/model_autoencoder.hdf5")